In [2]:
import pandas as pd
import requests
import numpy as np
import json
import asyncio
import dotenv
import boto3
import scrapy
import glob
import os
import logging
import asyncio
import psycopg2
import plotly.express as px
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from collections import defaultdict
from scrapy.crawler import CrawlerProcess, CrawlerRunner
from twisted.internet import asyncioreactor
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner






## Coordonnées GPS et METEO
Pour débuter le projet, nous allons récupérer les coordonnées géographiques (longitude et latitude) de chaque ville de notre liste en utilisant l’API Nominatim.
Dans un second temps, nous collecterons les données météorologiques correspondantes via l’API OpenWeatherMap.
Nous choisirons de conserver la température maximale, les precipitations et que l’indice de conditions météo (ensoleillé, pluvieux, nuageux, etc.).
L’ensemble de ces informations sera compilé dans un fichier unique csv.

In [ ]:

villes = {"Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"
}
headers = {"User-Agent": "KayakProject/1.0 (gdleds31@gmail.com)"}
data = []
for idx, ville in enumerate(villes):
    url = f"https://nominatim.openstreetmap.org/search?q={ville}&format=json&addressdetails=1&limit=1&polygon_svg=1"
    response = requests.get(url, headers=headers).json()
    if response:
        data.append({
            "ville": ville,
            "latitude": response[0]["lat"],
            "longitude": response[0]["lon"]
        })

df_villes = pd.DataFrame(data)


In [ ]:
# On vérifie les données récupérées
df_villes.info(), df_villes.head()

In [ ]:
df_villes.head()

### METEO - API OPENWEATHERMAP
Tout d'abord faisons une simple requête de nom des villes afin de voir si tout fonctionne bien.

In [20]:
API_KEY = '50354bff210b5943251ba7d75d2024ba'

city_data = []
for lat, lon in df_villes[["latitude", "longitude"]].values:
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&cnt=40&appid={API_KEY}&units=metric&lang=fr"
    response = requests.get(url).json()
    if response:
        city_data.append({"ville-1" : response['city']['name']})
city_data = pd.DataFrame(city_data)
df_villes1 = pd.concat([df_villes, city_data], axis=1)
df_villes1.head(35)

,ville,latitude,longitude,id,ville-1
0,Grenoble,45.1875602,5.7357819,0,La Tronche
1,Rouen,49.4404591,1.0939658,1,Rouen
2,Besancon,47.2380222,6.0243622,2,Besançon
3,Aix en Provence,43.5298424,5.4474738,3,Aix-en-Provence
4,Cassis,43.2140359,5.5396318,4,Cassis
5,Biarritz,43.4711437,-1.5527266,5,Biarritz
6,Toulouse,43.6044638,1.4442433,6,Toulouse
7,Nimes,43.8374249,4.3600687,7,Nîmes
8,Lyon,45.7578137,4.8320114,8,Vieux Lyon
9,Dijon,47.3215806,5.0414701,9,Larrey


In [21]:
# vérifions si nos données villes sont cohérentes
for ville in df_villes1['ville']:
    if ville == df_villes1['ville-1'].values[df_villes1['ville'] == ville][0]:
        pass
    else:
        print(f"Attention, la ville {ville} n'est pas cohérente avec {df_villes1['ville-1'].values[df_villes1['ville'] == ville][0]}")


Attention, la ville Grenoble n'est pas cohérente avec La Tronche
Attention, la ville Besancon n'est pas cohérente avec Besançon
Attention, la ville Aix en Provence n'est pas cohérente avec Aix-en-Provence
Attention, la ville Nimes n'est pas cohérente avec Nîmes
Attention, la ville Lyon n'est pas cohérente avec Vieux Lyon
Attention, la ville Dijon n'est pas cohérente avec Larrey
Attention, la ville Lille n'est pas cohérente avec La Madeleine
Attention, la ville Ariege n'est pas cohérente avec Larbont
Attention, la ville St Malo n'est pas cohérente avec Dufrost
Attention, la ville Uzes n'est pas cohérente avec Uzès
Attention, la ville Chateau du Haut Koenigsbourg n'est pas cohérente avec Saint-Hippolyte
Attention, la ville Le Havre n'est pas cohérente avec Havre-de-Grâce
Attention, la ville Bormes les Mimosas n'est pas cohérente avec Bormes-les-Mimosas
Attention, la ville Aigues Mortes n'est pas cohérente avec Aigues-Mortes
Attention, la ville Gorges du Verdon n'est pas cohérente avec Ba

Analyse des incohérences entre Nominatim et OpenWeatherMap

Lors de la comparaison des coordonnées GPS et des noms de villes renvoyés respectivement par Nominatim et OpenWeatherMap, nous avons observé plusieurs incohérences.

Dans la majorité des cas, il s’agit de simples différences d’orthographe ou d’usage des accents (par exemple Uzes vs Uzès, Besancon vs Besançon). Ces écarts sont mineurs et n’ont pas d’impact sur l’exploitation des données.

Dans d’autres cas, la localisation renvoyée correspond bien à la ville recherchée mais fait référence à un quartier ou à une zone voisine (par exemple Paris => Quartier du Palais-Royal, Lyon => Vieux Lyon). Ces variations restent acceptables car elles pointent bien sur la zone géographique attendue.

En revanche, deux cas nécessitent une correction spécifique :

Ariège : Nominatim interprète la requête comme le département et prend la ville la plus au centre de la région. Nous avons choisi de remplacer par Foix, chef-lieu de l’Ariège, afin d’obtenir une localisation cohérente.

St Malo : l’abréviation "St" est mal interprétée par OpenWeatherMap (Dufrost). Pour corriger ce problème, nous utiliserons la forme complète Saint Malo.


In [18]:
villes -= {"Ariege", "St Malo"}

villes |= {"Foix", "Saint Malo"}

In [ ]:
# On relance la récupération des coordonnées 
headers = {"User-Agent": "KayakProject/1.0 (gdleds31@gmail.com)"}
data = []
for idx, ville in enumerate(villes):
    url = f"https://nominatim.openstreetmap.org/search?q={ville}&format=json&addressdetails=1&limit=1&polygon_svg=1"
    response = requests.get(url, headers=headers).json()
    if response:
        data.append({
            "ville": ville,
            "latitude": response[0]["lat"],
            "longitude": response[0]["lon"]
        })

df_villes = pd.DataFrame(data)

# On crée une colonne id pour chaque ville
df_villes['id'] = pd.factorize(df_villes['ville'])[0]

# On récupère les données météo pour chaque ville
API_KEY = '50354bff210b5943251ba7d75d2024ba'

city_data = []
for lat, lon in df_villes[["latitude", "longitude"]].values:
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&cnt=40&appid={API_KEY}&units=metric&lang=fr"
    response = requests.get(url).json()
    if response:
        city_data.append({"ville-1" : response['city']['name']})
city_data = pd.DataFrame(city_data)
df_villes1 = pd.concat([df_villes, city_data], axis=1)
df_villes1.head(35)



,ville,latitude,longitude,id,ville-1
0,Le Havre,49.4938975,0.1079732,0,Havre-de-Grâce
1,Nimes,43.8374249,4.3600687,1,Nîmes
2,Saint Malo,48.6495180,-2.0260409,2,Saint-Malo
3,Besancon,47.2380222,6.0243622,3,Besançon
4,Bayonne,43.4945144,-1.4736657,4,Le Limpou
5,Lille,50.6365654,3.0635282,5,Lille
6,La Rochelle,46.1597320,-1.1515951,6,La Rochelle
7,Bormes les Mimosas,43.1506968,6.3419285,7,Bormes-les-Mimosas
8,Saintes Maries de la mer,43.4515922,4.4277202,8,Saintes-Maries-de-la-Mer
9,Mont Saint Michel,48.6359541,-1.5114600,9,Huisnes-sur-Mer


In [20]:
# vérifions si nos données villes sont cohérentes
for ville in df_villes1['ville']:
    if ville == df_villes1['ville-1'].values[df_villes1['ville'] == ville][0]:
        pass
    else:
        print(f"Attention, la ville {ville} n'est pas cohérente avec {df_villes1['ville-1'].values[df_villes1['ville'] == ville][0]}")

Attention, la ville Le Havre n'est pas cohérente avec Havre-de-Grâce
Attention, la ville Nimes n'est pas cohérente avec Nîmes
Attention, la ville Saint Malo n'est pas cohérente avec Saint-Malo
Attention, la ville Besancon n'est pas cohérente avec Besançon
Attention, la ville Bayonne n'est pas cohérente avec Le Limpou
Attention, la ville Bormes les Mimosas n'est pas cohérente avec Bormes-les-Mimosas
Attention, la ville Saintes Maries de la mer n'est pas cohérente avec Saintes-Maries-de-la-Mer
Attention, la ville Mont Saint Michel n'est pas cohérente avec Huisnes-sur-Mer
Attention, la ville Gorges du Verdon n'est pas cohérente avec Bauduen
Attention, la ville Uzes n'est pas cohérente avec Uzès
Attention, la ville Biarritz n'est pas cohérente avec La Négresse
Attention, la ville Lyon n'est pas cohérente avec Vieux Lyon
Attention, la ville Aigues Mortes n'est pas cohérente avec Aigues-Mortes
Attention, la ville Aix en Provence n'est pas cohérente avec Aix-en-Provence
Attention, la ville Ch

Nous allons récupérer les informations météo sur 5 jours via l’API OpenWeather. Pour chaque jour, nous prendrons uniquement les données de 9h (matin) et de 15h (après-midi). Nous nous intéresserons uniquement à la température maximale, aux précipitations et à la description du temps. Pour chaque type de données, nous créerons de nouvelles colonnes dans notre tableau. Enfin, nous fusionnerons ce tableau avec notre fichier contenant les coordonnées GPS des villes.

In [21]:
weather_data = []

# Boucle sur les villes
for lat, lon, city in df_villes[["latitude", "longitude","ville"]].values:
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&cnt=40&appid={API_KEY}&units=metric&lang=fr"
    response = requests.get(url).json()
    


    # Dictionnaire temporaire pour stocker les données par jour
    daily_data = {}

    for forecast in response["list"]:
        date = forecast["dt_txt"].split(" ")[0]
        hour = forecast["dt_txt"].split(" ")[1]

        # On s’intéresse uniquement à 09:00:00 et 15:00:00 pour avoir une météo matin et après midi
        if hour == "09:00:00" or hour == "15:00:00":
            temp_max = forecast["main"]["temp_max"]
            description = forecast["weather"][0]["description"]

            # Gestion des précipitations
            precipitation = 0
            if "rain" in forecast and "3h" in forecast["rain"]:
                precipitation = forecast["rain"]["3h"]
            elif "snow" in forecast and "3h" in forecast["snow"]:
                precipitation = forecast["snow"]["3h"]

            # Initialisation du dictionnaire si nouveau jour
            if date not in daily_data:
                daily_data[date] = {
                    "ville": city,
                    "date": date,
                    "temp_max_matin": None,
                    "temp_max_aprem": None,
                    "precip_matin": None,
                    "precip_aprem": None,
                    "desc_matin": None,
                    "desc_aprem": None
                }

            # Remplir selon l’heure
            if hour == "09:00:00":
                daily_data[date]["temp_max_matin"] = temp_max
                daily_data[date]["precip_matin"] = precipitation
                daily_data[date]["desc_matin"] = description
            elif hour == "15:00:00":
                daily_data[date]["temp_max_aprem"] = temp_max
                daily_data[date]["precip_aprem"] = precipitation
                daily_data[date]["desc_aprem"] = description

    # Ajoute les données du jour au tableau final
    weather_data.extend(daily_data.values())

# Convertir en DataFrame
df_meteo = pd.DataFrame(weather_data)

# # Joindre aux infos des villes si besoin
# df_ville_meteo = pd.merge(df_villes1, df_meteo, left_on="ville", right_on="ville", how="left")



In [22]:
# vérification des données fusionnées
display(df_meteo.head(5), df_meteo.isna().sum())

,ville,date,temp_max_matin,temp_max_aprem,precip_matin,precip_aprem,desc_matin,desc_aprem
0,Le Havre,2025-09-25,15.26,15.74,0.0,0.00,ciel dégagé,nuageux
1,Le Havre,2025-09-26,12.10,14.54,0.0,0.00,couvert,couvert
2,Le Havre,2025-09-27,12.50,16.03,0.0,0.00,nuageux,couvert
3,Le Havre,2025-09-28,14.43,15.57,0.0,0.13,couvert,légère pluie
4,Le Havre,2025-09-29,16.06,16.20,0.0,0.00,ciel dégagé,couvert


ville             0
date              0
temp_max_matin    0
temp_max_aprem    0
precip_matin      0
precip_aprem      0
desc_matin        0
desc_aprem        0
dtype: int64

## Données Hotels
Nous devons maintenant récupérer les données des hôtels présents dans chacune des villes de notre liste.
Pour cela, nous utilisons un script Python qui va scraper le site Booking.com.
Ce script génère un fichier JSON par ville, puis tous ces fichiers sont regroupés et consolidés en un seul fichier CSV.

In [ ]:
!python booking_final.py



🚀 Lancement batch 1: ['Mont Saint Michel', 'St Malo']

2025-09-06 09:42:00 [scrapy.utils.log] INFO: Scrapy 2.13.3 started (bot: scrapybot)
2025-09-06 09:42:00 [scrapy.utils.log] INFO: Versions:
{'lxml': '6.0.1',
 'libxml2': '2.14.5',
 'cssselect': '1.3.0',
 'parsel': '1.10.0',
 'w3lib': '2.3.1',
 'Twisted': '25.5.0',
 'Python': '3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]',
 'pyOpenSSL': '25.1.0 (OpenSSL 3.5.2 5 Aug 2025)',
 'cryptography': '45.0.7',
 'Platform': 'Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39'}
2025-09-06 09:42:00 [scrapy.addons] INFO: Enabled addons:
[]
2025-09-06 09:42:00 [scrapy.extensions.telnet] INFO: Telnet Password: e6a679d5ea30b424
2025-09-06 09:42:00 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle']
2025-09-06 09:42:00 [scrapy.craw

In [ ]:

# On fusionne les fichiers charger par scrapy
path = 'booking_results'
all_files = glob.glob(os.path.join(path, "*.json"))


list_of_dfs = []

# On lit chaque fichier JSON et on ajoute son contenu à la liste
for file in all_files:
    try:
        df = pd.read_json(file)  
        list_of_dfs.append(df)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier : {file}")
        print("Erreur :", e)

# On Concaténe tous les DataFrames en un seul et on crée un fichier CSV
df_hotels = pd.concat(list_of_dfs, ignore_index=True)

# On vérifie les données
display(df_hotels.head(), df_hotels.info(),df_hotels.isna().sum())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ville        875 non-null    object 
 1   name         875 non-null    object 
 2   description  875 non-null    object 
 3   url          875 non-null    object 
 4   rating       865 non-null    object 
 5   latitude     875 non-null    float64
 6   longitude    875 non-null    float64
dtypes: float64(2), object(5)
memory usage: 48.0+ KB


,ville,name,description,url,rating,latitude,longitude
0,Aigues Mortes,Résidence Odalys Fleur de Sel,"Situé à Aigues-Mortes, à 5 minutes à pied du c...",https://www.booking.com/hotel/fr/residence-oda...,"Avec une note de 8,4",43.574094,4.179246
1,Aigues Mortes,Résidence Odalys Le Mas des Flamants,Située à l'extérieur des fortifications d'Aigu...,https://www.booking.com/hotel/fr/residence-le-...,"Avec une note de 8,4",43.573794,4.180622
2,Aigues Mortes,La maison sur la place dans Aigues Mortes,"Situé à Aigues-Mortes, l’hébergement La maison...",https://www.booking.com/hotel/fr/la-maison-sur...,"Avec une note de 9,2",43.565873,4.192959
3,Aigues Mortes,Maison sol en ciel Petit Dejeuner et Parking i...,L’établissement Maison sol en ciel Petit Dejeu...,https://www.booking.com/hotel/fr/maison-sol-en...,"Avec une note de 9,9",43.564323,4.193398
4,Aigues Mortes,Marina Aigues Mortes 75m2,L’hébergement Marina Aigues Mortes 75m2 se sit...,https://www.booking.com/hotel/fr/marina-aigues...,"Avec une note de 9,6",43.574746,4.188398


None

ville           0
name            0
description     0
url             0
rating         10
latitude        0
longitude       0
dtype: int64

La colonne rating contient 10 valeurs manquantes. Après vérification sur Booking, il s’avère que ces hébergements sont trop récents et n’ont pas encore de note. Nous allons donc supprimer ces lignes de notre dataset.

In [ ]:
df_hotels.dropna(subset=['rating'], inplace=True)
display(df_hotels.head(), df_hotels.info(),df_hotels.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 865 entries, 0 to 874
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ville        865 non-null    object 
 1   name         865 non-null    object 
 2   description  865 non-null    object 
 3   url          865 non-null    object 
 4   rating       865 non-null    object 
 5   latitude     865 non-null    float64
 6   longitude    865 non-null    float64
dtypes: float64(2), object(5)
memory usage: 54.1+ KB


,ville,name,description,url,rating,latitude,longitude
0,Aigues Mortes,Résidence Odalys Fleur de Sel,"Situé à Aigues-Mortes, à 5 minutes à pied du c...",https://www.booking.com/hotel/fr/residence-oda...,"Avec une note de 8,4",43.574094,4.179246
1,Aigues Mortes,Résidence Odalys Le Mas des Flamants,Située à l'extérieur des fortifications d'Aigu...,https://www.booking.com/hotel/fr/residence-le-...,"Avec une note de 8,4",43.573794,4.180622
2,Aigues Mortes,La maison sur la place dans Aigues Mortes,"Situé à Aigues-Mortes, l’hébergement La maison...",https://www.booking.com/hotel/fr/la-maison-sur...,"Avec une note de 9,2",43.565873,4.192959
3,Aigues Mortes,Maison sol en ciel Petit Dejeuner et Parking i...,L’établissement Maison sol en ciel Petit Dejeu...,https://www.booking.com/hotel/fr/maison-sol-en...,"Avec une note de 9,9",43.564323,4.193398
4,Aigues Mortes,Marina Aigues Mortes 75m2,L’hébergement Marina Aigues Mortes 75m2 se sit...,https://www.booking.com/hotel/fr/marina-aigues...,"Avec une note de 9,6",43.574746,4.188398


None

ville          0
name           0
description    0
url            0
rating         0
latitude       0
longitude      0
dtype: int64

In [23]:
# Enregistrement des fichiers CSV en local
# df_hotels.to_csv('output/hotels.csv')
df_villes1.to_csv('output/villes2.csv')
df_meteo.to_csv('output/meteo2.csv')

#### Envoi des fichiers CSV sur notre S3

In [24]:
# chargement du fichier secret.env pour connecter à AWS S3
load_dotenv("secret.env")

AWS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")


s3 = boto3.client("s3", 
                  aws_access_key_id=AWS_KEY, 
                  aws_secret_access_key=AWS_SECRET)



In [25]:
# transférer les fichiers dans le bucket
# s3.upload_file("output/hotels.csv", BUCKET_NAME, "hotels.csv")
s3.upload_file('output/meteo2.csv', BUCKET_NAME, "meteo2.csv")
s3.upload_file('output/villes2.csv', BUCKET_NAME, "villes2.csv")

Maintenant, nous allons récupérer les fichiers depuis notre S3 afin de les charger dans notre base de données. Pour cela, nous utiliserons NeonDB, une base de données relationnelle PostgreSQL managée dans le cloud. Elle nous permettra de stocker, organiser et interroger efficacement les données que nous avons préalablement déposées dans S3.

In [72]:
# On charge les données depuis le S3
s3.download_file(BUCKET_NAME, "meteo2.csv", "input/meteoS3.csv")
s3.download_file(BUCKET_NAME, "villes2.csv", "input/villesS3.csv")
s3.download_file(BUCKET_NAME, "hotels.csv", "input/hotelS3.csv")
df_hotelS3 = pd.read_csv("input/hotelS3.csv")
df_meteoS3 = pd.read_csv("input/meteoS3.csv")
df_villesS3 = pd.read_csv("input/villesS3.csv")
# On vérifie si nos données sont bien chargées
# df_hotelS3.head(), df_meteoS3.head()

La colonne rating contient du texte avec la note intégrée. Nous allons créer une nouvelle colonne score qui contiendra uniquement la valeur numérique de la note, puis nous supprimerons la colonne rating devenue inutile.

In [ ]:
df_hotelS3['score']= df_hotelS3['rating'].str.split().str[4]
df_hotelS3['score'] = df_hotelS3['score'].str.replace(',', '.').astype(float)
df_hotelS3.drop('rating', axis=1, inplace=True)

In [75]:
# Suppression de la colonne Unnamed: 0 et 'ville-1'
df_villesS3.drop(['Unnamed: 0','ville-1'], axis=1, inplace=True)


In [76]:
# Connexion à la base de données PostgreSQL
load_dotenv("secret.env")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

# Créer l'engine SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")

In [77]:
# Suppession des tables si elles existent
with engine.connect() as connection:
    connection.execute(text("DROP TABLE IF EXISTS hotels2 CASCADE"))
    connection.execute(text("DROP TABLE IF EXISTS villes2 CASCADE"))
    connection.execute(text("DROP TABLE IF EXISTS meteo2 CASCADE"))

In [78]:


# Nous créeons les tables
create_table_villes2 = """
CREATE TABLE IF NOT EXISTS villes2(
    id INT PRIMARY KEY,
    ville VARCHAR(255),
    longitude DOUBLE PRECISION,
    latitude DOUBLE PRECISION
    );
"""

create_table_meteo2 = """
CREATE TABLE IF NOT EXISTS meteo2 (
    id_meteo SERIAL PRIMARY KEY,
    id INT REFERENCES villes2(id),
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    temp_max_matin FLOAT,
    temp_max_aprem FLOAT,
    precip_matin FLOAT,
    precip_aprem FLOAT,
    desc_matin VARCHAR(255),
    desc_aprem VARCHAR(255),
    date DATE
);
"""
create_table_hotels2 = """
CREATE TABLE IF NOT EXISTS hotels2 (
    id_hotels SERIAL PRIMARY KEY,
    id INT REFERENCES villes2(id),
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    name VARCHAR(255),
    description TEXT,
    url TEXT,
    score FLOAT 
    );
"""
# Exécuter la requête
with engine.connect() as connection:
    connection.execute(text(create_table_villes2))
    connection.execute(text(create_table_meteo2))
    connection.execute(text(create_table_hotels2))
  
    connection.commit()

print("Tables créées.")

Tables créées.


In [79]:
with engine.begin() as conn:
    conn.execute(text("TRUNCATE TABLE villes2 CASCADE"))
    # CASCADE pour vider aussi les tables qui dépendent de villes2 si nécessaire
    # si tu veux garder meteo2 et hotels2, enlève CASCADE


In [80]:
# Insertion de la table villes
df_villesS3.to_sql("villes2", engine, if_exists="append", index=False)

35

In [82]:
# merge pour insertion des clés étrangères
villes_mapping = pd.read_sql("SELECT id, ville FROM villes2", engine)
df_hotelS3 = df_hotelS3.merge(villes_mapping, on='ville', how='left')
df_meteoS3 = df_meteoS3.merge(villes_mapping, on="ville", how='left')

In [83]:
#supprimer la colonne ville avant insertion à la table
df_hotelS3.drop(columns=['ville'], inplace=True)



In [84]:
#supprimer la colonne ville et Unnamed: 0 avant insertion à la table
df_meteoS3.drop(columns=['ville','Unnamed: 0'], inplace=True)


In [85]:
# Nous envoyons nos données dans les tables
df_meteoS3.to_sql("meteo2", engine, if_exists="append", index=False)
df_hotelS3.to_sql("hotels2", engine, if_exists="append", index=False)

865

In [86]:
# Requête pour vérifier les données insérées
df_meteo2 =pd.read_sql("SELECT * FROM meteo2 LIMIT 10", con=engine)
print(f"Lecture depuis le fichier meteo :\n{df_meteo2}")

Lecture depuis le fichier meteo :
   id_meteo  id latitude longitude  temp_max_matin  temp_max_aprem  \
0       386   0     None      None           15.26           15.74   
1       387   0     None      None           12.10           14.54   
2       388   0     None      None           12.50           16.03   
3       389   0     None      None           14.43           15.57   
4       390   0     None      None           16.06           16.20   
5       391   1     None      None           15.99           20.04   
6       392   1     None      None           17.36           21.58   
7       393   1     None      None           16.53           23.11   
8       394   1     None      None           18.72           22.16   
9       395   1     None      None           19.22           24.29   

   precip_matin  precip_aprem   desc_matin    desc_aprem        date  
0           0.0          0.00  ciel dégagé       nuageux  2025-09-25  
1           0.0          0.00      couvert       couv

In [87]:
# Requête pour vérifier les données insérées
df_villes2= pd.read_sql("SELECT latitude FROM villes2 LIMIT 5", con=engine)
print(f"Lecture depuis le fichier meteo :\n{df_villes2}")

Lecture depuis le fichier meteo :
    latitude
0  49.493898
1  43.837425
2  48.649518
3  47.238022
4  43.494514


In [88]:
# Lire depuis hotels
df_hotels2 = pd.read_sql("SELECT * FROM hotels2 LIMIT 40", con=engine)
print(f"Lecture depuis le fichier hotels :\n{df_hotels2}")

Lecture depuis le fichier hotels :
    id_hotels  id   latitude  longitude  \
0        1731  20  43.574094   4.179246   
1        1732  20  43.573794   4.180622   
2        1733  20  43.565873   4.192959   
3        1734  20  43.564323   4.193398   
4        1735  20  43.574746   4.188398   
5        1736  20  43.570589   4.187651   
6        1737  20  43.565819   4.205353   
7        1738  20  43.565203   4.189902   
8        1739  20  43.573982   4.195183   
9        1740  20  43.586060   4.197451   
10       1741  20  43.565819   4.205353   
11       1742  20  43.568922   4.188115   
12       1743  20  43.567220   4.189344   
13       1744  20  43.575810   4.182881   
14       1745  20  43.561044   4.217806   
15       1746  20  43.572988   4.194825   
16       1747  20  43.573603   4.187765   
17       1748  20  43.571866   4.193662   
18       1749  20  43.569828   4.189679   
19       1750  20  43.562119   4.185999   
20       1751  20  43.574800   4.190900   
21       1752  20  

On nous a demandé d’identifier le Top 5 des destinations et de les représenter sur une carte.
Pour déterminer ce Top 5, nous avons pris en compte la température moyenne sur les 5 prochains jours ainsi que le niveau de précipitation le plus faible.
Ensuite, nous afficherons sur une autre carte le Top 20 des hôtels par ville, puis nous terminerons par la visualisation du Top 20 des hôtels situés dans les 5 destinations retenues.

In [89]:
# Les 5 villes les plus chaudes
query_cities_top5="""
WITH top_ville AS(
    SELECT m.id,
        AVG((m.temp_max_matin+m.temp_max_aprem)/2) AS temp_moy,
        AVG((m.precip_matin+m.precip_aprem)/2) AS precip_moy
    FROM meteo2 m
    GROUP BY m.id
)
SELECT v.ville,
        v.latitude,
        v.longitude,
        tp.temp_moy,
        tp.precip_moy
    FROM top_ville tp
    JOIN villes2 v ON tp.id=v.id
    ORDER BY tp.temp_moy DESC, tp.precip_moy ASC
    LIMIT 5;
"""

top5 = pd.read_sql(query_cities_top5, con=engine)
print(top5)
fig = px.scatter_map(top5, 
                    lat='latitude', 
                    lon='longitude', 
                    hover_name='ville',
                    hover_data={'temp_moy'},
                    color='temp_moy',
                    color_continuous_scale=("blue","orange","red"),
                    title="Top 5 villes les plus chaudes",
                    zoom=5,
                    # height=600
                    )
fig.update_layout(map_style="carto-positron")
fig.show()

         ville   latitude  longitude  temp_moy  precip_moy
0      Avignon  43.949249   4.805901    20.063       0.000
1        Nimes  43.837425   4.360069    19.900       0.000
2  Carcassonne  43.213036   2.349107    19.814       0.000
3    Collioure  42.525050   3.083155    19.681       0.034
4         Uzes  44.012128   4.419672    19.286       0.000


In [90]:
# Les 20 meilleurs hôtels par ville
query_hotel_top20_per_city = """
SELECT v.ville,
        h.name,
        h.latitude,
        h.longitude,
        h.score
FROM hotels2 h
JOIN villes2 v ON h.id=v.id
ORDER BY h.score DESC
LIMIT 20;
"""

top = pd.read_sql(query_hotel_top20_per_city, con=engine)
print(top)
fig = px.scatter_map(top, 
                    lat='latitude', 
                    lon='longitude', 
                    hover_name='name',
                    hover_data='score',
                    color='ville',
                    title="Top 20 hôtels par ville",
                    zoom=5,
                    height=600)
fig.update_layout(map_style="carto-positron")
fig.show()


                           ville  \
0                        Avignon   
1                    Carcassonne   
2   Chateau du Haut Koenigsbourg   
3       Saintes Maries de la mer   
4             Bormes les Mimosas   
5                         Bayeux   
6                           Uzes   
7              Mont Saint Michel   
8             Bormes les Mimosas   
9                      Eguisheim   
10                          Uzes   
11                 Aigues Mortes   
12                       Avignon   
13                        Bayeux   
14                 Aigues Mortes   
15                    Strasbourg   
16      Saintes Maries de la mer   
17                   Carcassonne   
18                          Foix   
19                         Nimes   

                                                 name   latitude  longitude  \
0                                     Le Clos Saluces  43.950193   4.810586   
1                               Le Belvédère St Gimer  43.206833   2.361293   
2     

In [91]:
# Les 20 meilleurs hôtels dans les 5 villes les plus chaudes
query_top_destination="""
WITH top5_villes AS (
    SELECT m.id,
           AVG((m.temp_max_matin + m.temp_max_aprem)/2) AS temp_moy,
           AVG((m.precip_matin + m.precip_aprem)/2) AS precip_moy
    FROM meteo2 m
    GROUP BY m.id
    ORDER BY temp_moy DESC,
             precip_moy ASC
    LIMIT 5
),hotels_rn AS (
    SELECT h.*,
            ROW_NUMBER() OVER (PARTITION BY h.id ORDER BY h.score DESC) AS rn
    FROM hotels2 h
)
SELECT v.ville,
       h.latitude,
       h.longitude,
       h.name,
       h.score,
       t5.temp_moy,
       t5.precip_moy
FROM hotels_rn h
JOIN top5_villes t5
  ON h.id = t5.id
JOIN villes2 v
  ON h.id = v.id
WHERE h.rn <= 20
ORDER BY v.ville, h.score DESC;
"""

top_dest = pd.read_sql(query_top_destination, con=engine)
print(top_dest)

fig = px.scatter_map(top_dest, 
                    lat='latitude', 
                    lon='longitude', 
                    hover_name='name',
                    hover_data={"ville": True, "score": True, "temp_moy": True},
                    color="temp_moy",  
                    color_continuous_scale=('blue','orange','red'),
                    title="TOP 20 des hotels dans le TOP 5 des villes les plus chaudes",
                    zoom=5,
                    height=600)
fig.update_layout(map_style="carto-positron")
fig.show()

      ville   latitude  longitude  \
0   Avignon  43.950193   4.810586   
1   Avignon  43.951293   4.806370   
2   Avignon  43.948867   4.809148   
3   Avignon  43.945103   4.822203   
4   Avignon  43.947607   4.807996   
..      ...        ...        ...   
95     Uzes  44.014362   4.410920   
96     Uzes  44.012514   4.418554   
97     Uzes  44.012464   4.421800   
98     Uzes  44.007718   4.408578   
99     Uzes  44.007783   4.407181   

                                                 name  score  temp_moy  \
0                                     Le Clos Saluces   10.0    20.063   
1   PLACE DU PALAIS DES PAPES 3 chambres Parking C...    9.9    20.063   
2                                        le Travertin    9.6    20.063   
3                       Villa Aigarden maison d'hôtes    9.6    20.063   
4                                          KAROUBA.31    9.5    20.063   
..                                                ...    ...       ...   
95                                   

In [92]:
query_top_destination2="""
WITH meteo_stats AS (
    SELECT m.id,
        AVG((m.temp_max_matin+m.temp_max_aprem)/2) AS temp_moy,
        AVG((m.precip_matin+m.precip_aprem)/2) AS precip_moy
    FROM meteo2 m
    GROUP BY m.id
        ),

top5_meteo AS (
    SELECT id,temp_moy, precip_moy
    FROM meteo_stats
    WHERE temp_moy < 15 AND precip_moy = 0
    ORDER BY temp_moy DESC, precip_moy ASC
    LIMIT 5
        ),   
hotels_rn AS (
    SELECT h.*,
            ROW_NUMBER() OVER (PARTITION BY h.id ORDER BY h.score DESC) AS rn
    FROM hotels2 h
        )
SELECT v.ville,
       h.latitude,
       h.longitude,
       h.name,
       h.score,
       t5.temp_moy,
       t5.precip_moy
FROM hotels_rn h
JOIN top5_meteo t5
  ON h.id = t5.id
JOIN villes2 v
  ON h.id = v.id
WHERE h.rn <= 20
ORDER BY v.ville, h.score DESC;
"""

top_dest2 = pd.read_sql(query_top_destination2, con=engine)
print(top_dest2)

fig = px.scatter_mapbox(top_dest2, 
                    lat='latitude', 
                    lon='longitude', 
                    hover_name='name',
                    hover_data={"ville": True, "score": True, "temp_moy": True},
                    color="temp_moy",  
                    color_continuous_scale=('blue','orange','red'),
                    title="TOP 20 des hotels dans le TOP 5 des villes ayant une temperature inferieur à 15°C et sans précipitations",
                    zoom=5,
                    height=600)
fig.update_layout(mapbox_style="carto-positron")
fig.show()

     ville   latitude  longitude  \
0   Amiens  49.887694   2.294524   
1   Amiens  49.894753   2.299982   
2   Amiens  49.886054   2.290221   
3   Amiens  49.891626   2.301684   
4   Amiens  49.896102   2.310790   
5   Amiens  49.896155   2.306368   
6   Amiens  49.897063   2.303700   
7   Amiens  49.888542   2.331349   
8   Amiens  49.891809   2.306885   
9   Amiens  49.897527   2.316416   
10  Amiens  49.874971   2.271423   
11  Amiens  49.887352   2.281246   
12  Amiens  49.894445   2.300942   
13  Amiens  49.902358   2.314285   
14  Amiens  49.879613   2.266651   
15  Amiens  49.891249   2.309530   
16  Amiens  49.896985   2.298736   
17  Amiens  49.891591   2.313573   
18  Amiens  49.889779   2.302499   
19  Amiens  49.891676   2.307754   
20   Lille  50.637541   3.076923   
21   Lille  50.635281   3.060162   
22   Lille  50.634074   3.064229   
23   Lille  50.628842   3.051979   
24   Lille  50.628842   3.051979   
25   Lille  50.634074   3.065783   
26   Lille  50.631178   3.06

/tmp/ipykernel_1389/2409259914.py:41: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [93]:
# les 10 meilleurs hotels
query_top_hotels="""
SELECT name, longitude, latitude, score
FROM hotels2
ORDER BY score DESC
LIMIT 10;
"""
q = pd.read_sql(query_top_hotels, con=engine)
print(q)

                                                name  longitude   latitude  \
0  Appartement 3 pièces 4 personne centre village...   4.431478  43.452181   
1  Appartement Uzès, piscine, climatisation, park...   4.410136  44.008163   
2                                 Au pied du Château   7.354826  48.248402   
3                  Gîte au pied du Mont Saint Michel  -1.484611  48.611982   
4                              Le Belvédère St Gimer   2.361293  43.206833   
5                                    Le Clos Saluces   4.810586  43.950193   
6                            LEPTILOFT BAYEUX Centre  -0.715019  49.281332   
7               Villa Le Pas de Courtin by Interhome   6.302437  43.151361   
8                     T3- Proche plages et commerces   6.357146  43.123297   
9  La Perle Rose Villa Standing 5 étoiles avec pi...   4.185999  43.562119   

   score  
0   10.0  
1   10.0  
2   10.0  
3   10.0  
4   10.0  
5   10.0  
6   10.0  
7   10.0  
8   10.0  
9    9.9  


In [94]:
# les villes les plus chaudes
query_hot_points = """
WITH hot AS (
    SELECT m.id,
           AVG((m.temp_max_matin + m.temp_max_aprem)/2) AS temp_moy
    FROM meteo2 m
    GROUP BY m.id 
)
SELECT v.ville,
       v.latitude,
       v.longitude,
       tm.temp_moy
FROM hot tm
JOIN villes2 v ON v.id=tm.id
ORDER BY tm.temp_moy DESC
LIMIT 20
;
"""
p_hot = pd.read_sql(query_hot_points, con=engine)
print(p_hot)


                       ville   latitude  longitude  temp_moy
0                    Avignon  43.949249   4.805901    20.063
1                      Nimes  43.837425   4.360069    19.900
2                Carcassonne  43.213036   2.349107    19.814
3                  Collioure  42.525050   3.083155    19.681
4                       Uzes  44.012128   4.419672    19.286
5                  Marseille  43.296174   5.369953    19.197
6                   Toulouse  43.604464   1.444243    19.003
7            Aix en Provence  43.529842   5.447474    18.854
8              Aigues Mortes  43.566152   4.191540    18.694
9         Bormes les Mimosas  43.150697   6.341928    18.574
10                      Foix  42.963900   1.605381    18.570
11  Saintes Maries de la mer  43.451592   4.427720    18.525
12                   Bayonne  43.494514  -1.473666    18.477
13                    Cassis  43.214036   5.539632    18.456
14                 Montauban  44.017584   1.354999    18.220
15                  Biar